## AIGCDetectBenchmark


## Install requirement

In [ ]:
!git clone https://github.com/vohoaidanh/ADOF.git

In [ ]:
%cd ADOF

In [ ]:
!git checkout exp/backbones

In [ ]:
!pip install timm

In [ ]:
!pip install -r requirements.txt
!pip install tensorboardX -q


In [ ]:
# Without google colab
!pip install gdown -q
!apt-get install -y unzip -q
!apt-get install -y zip -q
!pip install tensorboardX -q
!pip install regex -q
!pip install imageio -q
!pip install opencv-python -q
!apt-get install -y libgl1-mesa-glx -q
!pip install scikit-learn -q
!pip install scikit-image -q
!pip install ftfy -q
!pip install natsort -q
!pip install blobfile -q

## Download Data

In [ ]:
#Download Train, val set
import gdown
file_id = '1xHhCcCWtv57tnlAKh5sLSofuzcZcFfsP' #Progan train/val 4 class [car, cat, chair, horse]
#file_id = '1cyGbozooewAt6pNH9gw2_sDMM-Locqh2' #Progan train/val 4 class [car, cat, chair, horse]
destination = '/workspace/Progan_train.zip'  # Desired file name and extension
# Construct the download URL
url = f'https://drive.google.com/uc?id={file_id}'
# Download the file
gdown.download(url, destination, quiet=False)
#Unzip
!mkdir /workspace/datasets
!unzip -q $destination -d /workspace/datasets/ForenSynths_train

In [ ]:
#Download small Test set (this is a small testset just use for reference during training)
import gdown
file_id = '1-02FPvdTaQFDMatEvXAeKLCmnwW6RLx4' #test_set
destination = '/content/test_set.zip'  # Desired file name and extension
# Construct the download URL
url = f'https://drive.google.com/uc?id={file_id}'
# Download the file
gdown.download(url, destination, quiet=False)
#Unzip
!mkdir /content/datasets
!unzip -q $destination -d /content/datasets/ForenSynths_train/test

## Train

In [ ]:
#experiment-01-no-filter
!find /workspace/datasets -type d -name "*ipynb*" -exec rm -r {} +
!python train.py \
--name adof-fastvit_sa24- \
--dataroot /workspace/datasets/ForenSynths_train \
--num_thread 2 \
--classes car,cat,chair,horse --batch_size 32 --delr_freq 5 --lr 0.0002 --niter 5

## Evaluation

In [ ]:
# You should update the test set directory in the "test.py" file.
!find /workspace/datasets -type d -name "*ipynb*" -exec rm -r {} +
!python test.py \
--model_path /workspace/model_vit_base_patch16_224_best_3.pth  \
--num_thread 4 \
--batch_size 32

In [ ]:
# You should update the test set directory in the "test.py" file.
!find /workspace/datasets -type d -name "*ipynb*" -exec rm -r {} +
!python test.py \
--model_path /workspace/model_vit_base_patch16_224_best_4-df93.pth  \
--num_thread 4 \
--batch_size 32

In [ ]:
# You should update the test set directory in the "test.py" file.
!find /workspace/datasets -type d -name "*ipynb*" -exec rm -r {} +
!python test.py \
--model_path /workspace/model_vit_base_patch16_224_best_4-df93.pth  \
--num_thread 4 \
--batch_size 32